In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers


In [2]:
%run utils.ipynb
%run trainingUtils.ipynb

In [25]:
class ANNTool():
    def __init__(self, input_shape=(128, 128, 3), epochs=100, activation='relu',
                 loss='binary_crossentropy', optimizer='adam'):
        self.input_shape = input_shape
        self.epochs = epochs
        self.activation = activation
        self.loss = loss
        self.optimizer = optimizer
        self.train_inputs = None
        self.train_outputs = None

    def train_classifier(self, train_inputs, train_outputs, epochs=None, activation=None, loss=None, optimizer=None):
        if epochs is not None:
            self.epochs = epochs
        if activation is not None:
            self.activation = activation
        if loss is not None:
            self.loss = loss
        if optimizer is not None:
            self.optimizer = optimizer
        self.train_inputs = train_inputs
        self.train_outputs = train_outputs
        classifier = tf.keras.Sequential([
            tf.keras.layers.InputLayer(shape=self.input_shape),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128, activation=self.activation),
            tf.keras.layers.Dense(64, activation=self.activation),
            tf.keras.layers.Dense(32, activation=self.activation),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])
        classifier.compile(optimizer=self.optimizer, loss=self.loss, metrics=['accuracy'])
        classifier.fit(train_inputs, train_outputs, epochs=self.epochs)
        return classifier

    def get_predicted_data(self,test_inputs):
        classifier = self.train_classifier(self.train_inputs, self.train_outputs)
        computed_outputs = classifier.predict(test_inputs)
        computed_outputs = np.round(computed_outputs)
        return computed_outputs

In [31]:
def run_hyperparameter_testing_ANN(train_inputs, train_outputs, test_inputs, test_outputs):
    hyperparameters = [
        {'epochs': 50, 'activation': 'relu', 'loss': 'binary_crossentropy', 'optimizer': 'adam'},
        {'epochs': 100, 'activation': 'relu', 'loss': 'binary_crossentropy', 'optimizer': 'sgd'},
        {'epochs': 100, 'activation': 'tanh', 'loss': 'mean_squared_error', 'optimizer': 'adam'},
        {'epochs': 50, 'activation': 'tanh', 'loss': 'mean_squared_error', 'optimizer': 'sgd'},
        {'epochs': 50, 'activation': 'relu', 'loss': 'mean_squared_error', 'optimizer': 'adam'},
        {'epochs': 100, 'activation': 'relu', 'loss': 'mean_squared_error', 'optimizer': 'sgd'},
        {'epochs': 50, 'activation': 'tanh', 'loss': 'binary_crossentropy', 'optimizer': 'adam'},
        {'epochs': 100, 'activation': 'tanh', 'loss': 'binary_crossentropy', 'optimizer': 'sgd'}
    ]
    train_inputs = np.array(train_inputs)
    train_outputs = np.array(train_outputs)
    test_inputs = np.array(test_inputs)
    test_outputs = np.array(test_outputs)
    results = []
    for config in hyperparameters:
        ann_tool = ANNTool()
        print(f"Training with config: {config}")
        model = ann_tool.train_classifier(
            train_inputs, train_outputs,
            epochs=config['epochs'],
            activation=config['activation'],
            loss=config['loss'],
            optimizer=config['optimizer']
        )

        predicted_outputs = ann_tool.get_predicted_data(test_inputs)
        accuracy = np.mean(predicted_outputs == test_outputs)
        print(f"Test Accuracy: {accuracy}")
        results.append((config, accuracy))

    return results


In [30]:
def with_ANNTool(train_inputs,train_outputs,test_inputs,test_outputs,output_names=['no_sepia','sepia']):
    train_inputs = np.array(train_inputs)
    train_outputs = np.array(train_outputs)
    test_inputs = np.array(test_inputs)
    test_outputs = np.array(test_outputs)
    try:
        annTool = ANNTool()
        annTool.train_classifier(train_inputs[:1], train_outputs[:1])
    except Exception as e:
        print("Error with a single data point:", e)
        print("Data point shape:", train_inputs[:1].shape)
        print("Label shape:", train_outputs[:1].shape)

    try:
        annTool = ANNTool()
        annTool.train_classifier(train_inputs, train_outputs)
    except Exception as e:
        print("An error occurred:", e)
    # Optionally, add more diagnostic print statements here

    computed_outputs=annTool.get_predicted_data(test_inputs)
    return computed_outputs
    
    